In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import geopandas as gpd
#import geopy?
%matplotlib inline

In [2]:
acs_datapath = "data_v2/Dept_37-00049/37-00049_ACS_data/"

In [3]:
education_data = "37-00049_ACS_education-attainment/ACS_16_5YR_S1501_with_ann.csv"

In [4]:
education_df = pd.read_csv(acs_datapath + education_data, header = 0, skiprows=[1])
sub_header = pd.read_csv(acs_datapath + education_data, header = 1).keys()

In [5]:
education_df.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC03_EST_VC02,HC03_MOE_VC02,HC04_EST_VC02,...,HC02_EST_VC85,HC02_MOE_VC85,HC03_EST_VC85,HC03_MOE_VC85,HC04_EST_VC85,HC04_MOE_VC85,HC05_EST_VC85,HC05_MOE_VC85,HC06_EST_VC85,HC06_MOE_VC85
0,1400000US48113000100,48113000100,"Census Tract 1, Dallas County, Texas",195,101,(X),(X),90,69,(X),...,(X),(X),178750,97008,(X),(X),91507,10438,(X),(X)
1,1400000US48113000201,48113000201,"Census Tract 2.01, Dallas County, Texas",36,25,(X),(X),36,25,(X),...,(X),(X),128182,33333,(X),(X),76875,16293,(X),(X)
2,1400000US48113000202,48113000202,"Census Tract 2.02, Dallas County, Texas",217,95,(X),(X),82,86,(X),...,(X),(X),101824,14028,(X),(X),57813,13941,(X),(X)
3,1400000US48113000300,48113000300,"Census Tract 3, Dallas County, Texas",262,176,(X),(X),176,138,(X),...,(X),(X),71250,36939,(X),(X),54375,21490,(X),(X)
4,1400000US48113000401,48113000401,"Census Tract 4.01, Dallas County, Texas",740,194,(X),(X),283,113,(X),...,(X),(X),58125,21588,(X),(X),55735,4681,(X),(X)


In [6]:
FIPS_state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

In [7]:
tx_census_shape_dir = "census_shape/tl_2016_48_tract/"
tx_census_shp = "tl_2016_48_tract.shp"

In [8]:
tx_census_shp_df = gpd.read_file(tx_census_shape_dir + tx_census_shp)
tx_census_shp_df.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,48,201,542301,48201542301,5423.01,Census Tract 5423.01,G5020,S,5956745,15456,+29.8204990,-095.7316493,"POLYGON ((-95.75223799999999 29.831686, -95.75..."
1,48,201,552601,48201552601,5526.01,Census Tract 5526.01,G5020,S,2911911,4263,+29.9629845,-095.5679180,"POLYGON ((-95.585241 29.966226, -95.5852209999..."
2,48,157,673800,48157673800,6738,Census Tract 6738,G5020,S,14100089,1008190,+29.5874575,-095.7125243,"POLYGON ((-95.758386 29.583531, -95.757993 29...."
3,48,157,674000,48157674000,6740,Census Tract 6740,G5020,S,4826110,65291,+29.5869509,-095.6224534,"POLYGON ((-95.63467299999999 29.589525, -95.63..."
4,48,157,674100,48157674100,6741,Census Tract 6741,G5020,S,4170610,0,+29.5791692,-095.5995891,"POLYGON ((-95.614667 29.578282, -95.6133859999..."


In [9]:
tx_census_shp_df.keys()

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [10]:
for v in education_df['GEO.id2'].values:
    found = False
    for r in tx_census_shp_df['GEOID'].values:
        if str(v)[:11] == str(r)[:11]:
            print("yeet", v)
            found = True
            break
    if not found:
        print("nawp", v)

yeet 48113000100
yeet 48113000201
yeet 48113000202
yeet 48113000300
yeet 48113000401
yeet 48113000404
yeet 48113000405
yeet 48113000406
yeet 48113000500
yeet 48113000601
yeet 48113000603
yeet 48113000605
yeet 48113000606
yeet 48113000701
yeet 48113000702
yeet 48113000800
yeet 48113000900
yeet 48113001001
yeet 48113001002
yeet 48113001101
yeet 48113001102
yeet 48113001202
yeet 48113001203
yeet 48113001204
yeet 48113001301
yeet 48113001302
yeet 48113001400
yeet 48113001502
yeet 48113001503
yeet 48113001504
yeet 48113001600
yeet 48113001701
yeet 48113001703
yeet 48113001704
yeet 48113001800
yeet 48113001900
yeet 48113002000
yeet 48113002100
yeet 48113002200
yeet 48113002400
yeet 48113002500
yeet 48113002701
yeet 48113002702
yeet 48113003101
yeet 48113003400
yeet 48113003700
yeet 48113003800
yeet 48113003901
yeet 48113003902
yeet 48113004000
yeet 48113004100
yeet 48113004201
yeet 48113004202
yeet 48113004300
yeet 48113004400
yeet 48113004500
yeet 48113004600
yeet 48113004700
yeet 481130048

In [22]:
test = tx_census_shp_df.iloc[[0]][['geometry']]
test

,geometry
0,"POLYGON ((-95.75223799999999 29.831686, -95.75..."


In [24]:
mapa = folium.Map([29.8204990, -095.7316493], height=400, zoom_start=10, tiles='Stamen Toner')
folium.GeoJson(test).add_to(mapa)
mapa